<a href="https://colab.research.google.com/github/Falconwatch/hackaton/blob/master/Preprocessor_Realisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Реализация ядра

In [0]:
import numpy as np
class Preporcessor:
  def __init__(self, filters):
    self._filters=filters

  
  def preprocess(self, initinal_strings):
    result= list()
    #входной массив обрабатываем поэлементно
    for initinal_string in initinal_strings:
      #если вдруг есть
      result.append(self._process_string(initinal_string))
    return np.concatenate(result)

  def _process_string(self, adress_string):
    result = list()
    #сплтим по переводу строки, если есть
    addresses = [s.lower() for s in adress_string.split('\n')]
    for address in addresses:
      #разобьём адре сна кусочки по запятым
      address_parts = [ap.strip() for ap in address.split(',')]
      for i in range(len(address_parts)):
        for one_filter in self._filters:
          address_parts[i] = one_filter(address_parts[i])

      address_fixed = ','.join([ap for ap in address_parts if len(ap)>0])
      #здесь вызываем поочердёно фильтры, лежащие в self._filters
      result.append(address_fixed)
   
    return result







#переводит римские числа в арабские
def filter_roman(input_string):
  def roman_to_int(s):
    if len(s)==0:
      return ''
    rom_val = {'i': 1, 'v': 5, 'x': 10, 'l': 50, 'c': 100, 'd': 500, 'm': 1000}
    int_val = 0
    for i in range(len(s)):
      if s[i] not in rom_val:
        return s
      if i > 0 and rom_val[s[i]] > rom_val[s[i - 1]]:
        int_val += rom_val[s[i]] - 2 * rom_val[s[i - 1]]
      else:
        int_val += rom_val[s[i]]
    return str(int_val)

  splited_string = input_string.split(' ')

  for i in range(len(splited_string)):
    splited_string[i] = roman_to_int(splited_string[i] ) 

  result = ' '.join(splited_string)
  return result

# фильтруем названия станций метро и мцк
def filter_stations(input_string):
  splited_string = input_string.split(',')
  for i in range(len(splited_string)):
    w = splited_string[i]
    if (w[:2]=='м.') or (w[:3]==' м.') or (w[-2:]==' м') or ('мцк' in w) or (w[-2:]=='ст.') :
      splited_string[i]=''
  return ''.join(splited_string)


# убираем плохие символы из строки
def filter_bad_signs(input_string):
  input_string = input_string.replace('№','')
  input_string = input_string.replace('/','\\')
  input_string = input_string.replace(',,',',')
  return input_string

# фиксим аббревиатуры
def filter_abb(input_string):
  input_string = input_string.replace('р-он','район')
  input_string = input_string.replace('р-н','район')
  input_string = input_string.replace('пом.','помещение')
  input_string = input_string.replace('пом ','помещение')
  input_string = input_string.replace('корп.','корпус')
  input_string = input_string.replace('корп ','корпус')
  input_string = input_string.replace('д. ','дом')
  input_string = input_string.replace('д ','дом')
  return input_string
    
def filter_exclam(input_string):
  input_string = input_string.replace('!-', '1-')
  return input_string

In [13]:
data =['string1 example', 'string2\nкорп. ivx', 'м. войковская, дом №2', '!-я владимирская', 'viii, asd']

p = Preporcessor([filter_exclam, filter_roman, filter_bad_signs, filter_abb, filter_stations])
p.preprocess(data)

array(['string1 example', 'string2', 'корпус 4', 'дом 2',
       '1-я владимирская', '8,asd'], dtype='<U16')

# Работа

In [0]:
import pandas as pd

In [5]:
bads = pd.read_parquet('bads_fixed.parquet')
bads.head()

,id,address
0,20000001,"115280, г. Москва, ул. Восточная, д. 9 этаж 1 ..."
1,20000002,"115280, г. Москва, ул. Ленинская Слобода, д. 2..."
2,20000003,"117105, г. Москва, шоссе Варшавское, д. 1 стр...."
3,20000004,"117321, г. Москва, ул. Профсоюзная, д. 128, ко..."
4,20000005,"121357, г. Москва, ул. Нежинская, д. 1 корп. 4..."


In [0]:
fixed_addresses = p.preprocess(bads['address'].values)

In [7]:
fixed_addresses

array(['115280, г. москва, ул. восточная, д.9 этаж 1 помещение 4 ком. 1',
       '115280, г. москва, ул. ленинская слобода, д.26 помещение 13 ком. 27',
       '117105, г. москва, шоссе варшавское, д.1 стр. 1-2 ком. 3б', ...,
       'шоссе энтузиастов, д.2, стр. 44, ст. москва-товарная-курская',
       'шоссе энтузиастов, д.2, стр. 9, ст. москва-товарная-курская',
       'ывы, г.москва, аллея. жемчуговой, дом 1, корпус 1'], dtype='<U307')

In [90]:
bads.loc[0, 'address']

'115280, г. Москва, ул. Восточная, д. 9 этаж 1 пом. IV ком. 1'

In [0]:
pd.DataFrame(fixed_addresses).to_csv('fixed_addresses.csv')


In [0]:
# Реализация оболочки